<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>
</a>
<h1 style="line-height: 1.4;"><font color="#76b900"><b>Building Agentic AI Applications with LLMs</h1>
<h2><b>Notebook 1:</b> Making A Simple Agent</h2>
<br>

**Hello, and welcome to the first notebook of the course!**

We will use this opportunity to introduce some starting tools to build a simple chat system and will contextualize their place within the agent classification space. Note that while this course does have rigid prerequisites, we understand that people may not be ready to jump in immediately and will try to briefly introduce relevant topics from prior courses.

### **Learning Objectives:**

**In this notebook, we will:**
- Gain a working understanding of the term "agent," and understand why it is once again gaining traction.
- Explore the course primitives, including the NIM Llama model running in the background of this environment.
- Make a simple chatbot, followed by a simple multi-agent system to allow for multi-turn multi-persona dialog.

<hr><br>

## **Part 1:** Boiling Down Agents

**In the lecture, we defined an agent as an entity among entities that exists and functions in an environment.** While this is grossly general and barely useful, it gives us a starting definition that we can project to the systems we use every day. Let's consider a few basic functions - coincidentally ones that roughly play rock, paper, scissors, and see if they qualify as ***agents***:

In [1]:
from random import randint

def greet(state):
    return print("Let's play a nice game of Rock/Paper/Scissors") or "nice"

def play(state):
    match randint(1, 3):
        case 1: return print("I choose rock") or "rock"
        case 2: return print("I choose paper") or "paper"
        case 3: return print("I choose scissors") or "scissors"

def judge(state):
    play_pair = state.get("my_play"), state.get("your_play")
    options = "rock", "paper", "scissors"
    ## Create pairs of options such as [(o1, o2), (o2, o3), (o3, o1)]
    loss_pairs = [(o1, o2) for o1, o2 in zip(options, options[1:] + options[:1])]
    ## Create pairs of options such as [(o2, o1), (o3, o2), (o1, o3)]
    win_pairs  = [(o2, o1) for o1, o2 in loss_pairs]
    if play_pair in loss_pairs:
        return print("I lost :(") or "user_wins"
    if play_pair in win_pairs:
        return print("I win :)") or "ai_wins"
    return print("It's a tie!") or "everyone_wins"

state = {}
state["my_tone"] = greet(state)
state["my_play"] = play(state)
state["your_play"] = input("Your Play").strip() or print("You Said: ", end="") or play(state)
state["result"] = judge(state)

print(state)

Let's play a nice game of Rock/Paper/Scissors
I choose paper


Your Play rock


I win :)
{'my_tone': 'nice', 'my_play': 'paper', 'your_play': 'rock', 'result': 'ai_wins'}


<br>

Together, they trivially define a computer program and technically interact with an environment of sorts:
- The **computer** renders the user interface for the human to interact with.
- The **Jupyter cell** stores lines of code which help to define a control flow that executes when the system runs.
- The **Python environment** stores variables, including function and state, and even the output buffer that gets rendered for the user.
- The **state dictionary** stores a state that can be written to.
- The **functions** take in the state dictionary, possibly act on it, and print/return values which may or may not be honored.
- ... so on and so forth.

There are obviously arbitrarily many things at play that contribute to the state of this system and that of the larger surrounding world, and yet nothing here nor there fully considers or even understands all of them. **All that matters is what's locally perceived, and this local perception drives local actions.** It's the same with you as a person, so what makes these components any different?

Well, the main difference here is that these components *do not feel* like they are meaningfully percieving the environment and intentionally choosing their actions. Put another way:
- The decomposition of a complex problem into modules of state and functionality glued together with some control flow defines good software engineering...
- But the *feeling* that components have the choice to do things and are driven by some tangible objective define our intuitive *agent* in a human sense. 

Since humans interact with the environment through the local perception of senses and reason about it semantically (through "thought" and "meaning"), an agent system that interacts with humans would need to either look and act in our shared physical space as a **physical agent**, or communicate like a human or persona would through a limited interface as a **digital agent**. But if it is to function *alongside* humans and *think* like a human, it would need to:
- At least be able to sustain some notion of internal thought and local perspective.
- Have some understanding of its environment and the notion of "goals" and "tasks."
- Be able to communicate through an interface that can be understood by a human.

These are all concepts that float around in **semantic space** - they have "meaning" and "causality" and "implications", and can be interpretted by humans and even algorithms when organized correctly - so we will need to be able to model these semantic concepts and create mappings from semantically-dense inputs to semantically-dense outputs. This is exactly where large language models come in.

<hr><br>

## **Part 2:** Semantic Reasoning with Technology

In most cases, software is programmed into intuitive modules that can be built upon to make complex systems. Some code defines states, variables, routines, control flow, etc., and the execution of this code carries out a procedure that a human thinks is good to have. The components are described, have meaning in their construction and function, and piece together logically because the developer decided to put them that way or because the structure emerged otherwise:

```python
from math import sqrt                             ## Import of complex environment resources

def fib(n):                                       ## Function to describe and encapsulate
    """Closed-form fibonacci via golden ratio"""  ## Semantic description to simplify
    return round(((1 + sqrt(5))/2)**n / sqrt(5))  ## Repeatable operation that users need not know

for i in range(10):                               ## Human-specified control flow
    print(fib(i))
```

With large language models trained on a giant repository of data, we can model the mapping from a semantically-meaningful input to a semantically-meaningful output with the power of inference.

**Specifically, the two main models we will care about are:**
- **Encoding Model:** $Enc: X \to R^{n}$, which maps input that has intuitive explicit form (i.e. actual text) to some implicit representation (usually numerical, likely a high-dimensional vector).
- **Decoding Model:** $Dec: R^{n}\cup X \to Y$, which maps input from some representation (maybe vector, maybe explicit) into some explicit representation.

These are highly-general constructs and various architectures can be made to implement them. For example, you may be familiar with the following formulations:
- **Text-Generating LLM:** $text \to text$ might be implemented with a forecasting model that is trained to predict one token after another. For example, $P(t_{m..m+n} | t_{0..m-1})$ might generate a series of $n$ tokens (substrings) from $m$ tokens by iterating on $P(t_{i} | t_{0..i-1})$ starting at $i=m$.
- **Vision LM:** $\{text, image\} \to text$ might be implemented as $Dec(Enc_1(text), Enc_2(image))$ where $Dec$ is has viable architecture for sequence modeling and $Enc_1/Enc_2$ just projects the natural inputs into a latent form.
- **Diffusion Model:** $\{text\} \to image$ might be implemented as $Dec(...(Dec(Dec(\xi_0))...)$ where $Dec$ iteratively denoises from a canvas of noise while also taking in some encoding $Enc(text)$ as conditioning.

For most of this course, we will mainly rely on a decoder-style (implied autoregressive) large language model which is running perpetually in the background of this environment. We can connect to one such model using the interface below, and can experiment with it using a [**LangChain LLM client developed by NVIDIA**](https://python.langchain.com/docs/integrations/chat/nvidia_ai_endpoints/) - which is really just a client that works with any OpenAI-style LLM endpoint with a few extra conveniences.

In [2]:
from langchain_nvidia import ChatNVIDIA
from langchain_core.messages import convert_to_messages
## Uncomment to list available models
# model_options = [m.id for m in ChatNVIDIA.get_available_models()]
# print(model_options)

## For the course, feel free to use any of these options:
# llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct", base_url="http://llm_client:9000/v1")
llm = ChatNVIDIA(model="nvidia/llama-3.1-nemotron-nano-8b-v1", base_url="http://llm_client:9000/v1")

This model, which is a [**Llama-8B-3.1-Instruct NIM-hosted model**](https://build.nvidia.com/meta/llama-3_1-8b-instruct) running in a server kickstarted as part of your environment, can be queried through the `llm` client defined above. We can send a single request to the model as follows, either with a single response which gets delivered all at once or a streamed response that creates a generator and outputs as tokens are produced.

In [3]:
%%time
print("[SINGLE RESPONSE]")
print(llm.invoke("Hello World").content)

[SINGLE RESPONSE]
Greetings! It seems like you've just started a new conversation here. I'm here to help. What can I do for you today? You've mentioned "Hello World". Should I just respond with a simple "Hello world!" or is there something specific you'd like me to assist you with?
CPU times: user 3.92 ms, sys: 3.95 ms, total: 7.87 ms
Wall time: 1.17 s


In [4]:
%%time
print("[STREAMED RESPONSE]")
for chunk in llm.stream("Hello world"):
    print(chunk.content, end="", flush=True)

[STREAMED RESPONSE]
Greetings! It seems like you've made a classic and straightforward program. "Hello world" is an iconic Python statement frequently used to introduce Python programming concepts.

Here's a confirmation that your code has executed as expected:

**System output:**
```
Hello world!
```
Is there anything particular about this program that you would like to discuss or further explore? For instance, you might be interested in learning how to create a "Goodbye world" program, how to customize the text within a "Hello world" program, or other variations within the scope of Python basics. Let me know if there's anything else you need help with!CPU times: user 140 ms, sys: 43.7 ms, total: 184 ms
Wall time: 1.89 s


In [5]:
%%time
print("[SINGLE RESPONSE]")
print(llm.invoke("Hello World").content)

[SINGLE RESPONSE]
Greetings! It seems like you've just started a new conversation here. I'm here to help. What can I do for you today? You've mentioned "Hello World". Should I just respond with a simple "Hello world!" or is there something specific you'd like me to assist you with?
CPU times: user 7.39 ms, sys: 468 μs, total: 7.86 ms
Wall time: 1.16 s


**From a technical perspective,** Between this simple request and the simple response lies layers of abstraction which include:
- A network request sent out to the `llm_client` microservice running a FastAPI router service.
- A network request sent out to a `nim` microservice running another FastAPI service and hosting a VLLM/Triton-backed model downloaded from a model registry.
- An insertion of the inputs into some prompt template that the model was actually trained for.
- A tokenization of the input from the templated string into a sequence of classes using something resembling the transformers preprocessing pipeline.
- An embedding of the inputted sequence of classes into some latent form using an embedding routine.
- A propogation of the input embeddings through a transformer-backed architecture to progressively convert the input embeddings into the output embeddings.
- And a progressive decoding of next tokens, sampled from the predicted probability over all token options, one at a time, until a stop token is generated.
- ... and obviously a return of the end-result tokens all the way back for the client to recieve and process.

**From our perspective,** our client facilitated the connection to a large language model through a network interface to - at minimum - send out a well-formatted request and accept a well-formatted response, as shown below:

In [6]:
llm._client.last_inputs

{'url': 'http://llm_client:9000/v1/chat/completions',
 'headers': {'Accept': 'application/json',
  'Authorization': 'Bearer **********',
  'User-Agent': 'langchain-nvidia-ai-endpoints',
  'X-BILLING-SOURCE': 'langchain-nvidia-ai-endpoints'},
 'json': {'messages': [{'role': 'user', 'content': 'Hello World'}],
  'model': 'nvidia/llama-3.1-nemotron-nano-8b-v1',
  'max_tokens': 1024,
  'stream': False}}

In [7]:
## Note, the client does not attempt to log 
llm._client.last_response.json()

{'id': 'chat-df62b09915e046f8bc5d0313df0a5713',
 'object': 'chat.completion',
 'created': 1770840701,
 'model': 'nvidia/llama-3.1-nemotron-nano-8b-v1',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': 'Greetings! It seems like you\'ve just started a new conversation here. I\'m here to help. What can I do for you today? You\'ve mentioned "Hello World". Should I just respond with a simple "Hello world!" or is there something specific you\'d like me to assist you with?'},
   'logprobs': None,
   'finish_reason': 'stop',
   'stop_reason': None}],
 'usage': {'prompt_tokens': 17, 'total_tokens': 77, 'completion_tokens': 60},
 'prompt_logprobs': None}

<br>

**Is this model inherently "thinking?"** Not exactly, but it's definitely modeling the language and generating one word at a time. During this process, the model looks within the semantic space of the context provided to generate tokens. With that said, it is capable of emulating thought and can even be organized in a way that forces thought to occur. ***More on that later.***

**Does this mean this model is an "agent?"** Also not exactly. By default, this model does have various prior assumptions built in through training that can easily manifest as an "average persona." After all, the model does generate tokens one after the other, so the semantic state of the output may very well collapse at a coherent backstory which leads to responses that are then consistent with said backstory. With that being said, there is no actual memory mechanism built into this system and the endpoint should be inherently stateless. 

We can send some requests to the model to see how it works below:

In [8]:
from langchain_nvidia import NVIDIA

## This is a more typical interface which accepts chat messages (or implicitly creates them)
print("Trying out some different /chat/completions sampling")
print("[A]", llm.bind(seed=42, stop="\n").invoke("Hello world").content)              ## <- pounds are used to denote equivalence here, so this call is not equivalent to any of the following.
print("[B]", llm.bind(seed=12, stop="\n").invoke("Hello world").content)              ### Changing the seed changes the sampling. This is usually subtle. 
print("[B]", llm.bind(seed=12, stop="\n").invoke("Hello world").content)              ### Same seed + same input = same sampling.
print("[B]", llm.bind(seed=12, stop="\n").invoke([("user", "Hello world")]).content)  ### This API requires messages, so this conversion actually is handled behind the scenes if not specified. 
print("[C]", llm.bind(seed=12, stop="\n").invoke("Hello world!").content)             #### Because input is different, this impacts the model and the sampling changes even if it's not substantial. 
print("[D]", llm.bind(seed=12, stop="\n").invoke("Hemlo wordly!").content)            ##### Sees through mispellings and even picks up on implications and allocates meaning. 

## This queries the underlying model using the completions API
completion_llm = NVIDIA(model="nvidia/mistral-nemo-minitron-8b-base", base_url="http://llm_client:9000/v1")
print("\nTrying out some different `/completions` sampling. Supported by NIMs, hidden by build.nvidia.com unless typical-use.")
print(f"Models with /completions as typical-use:")
print(*[f" - {repr(m)}" for m in completion_llm.get_available_models()], sep="\n")
print("\n[Hello world]" + completion_llm.bind(seed=42, max_tokens=20).invoke("Hello world").replace("\n", " ")) ######
print("\n[Hello world]" + completion_llm.bind(seed=12, max_tokens=20).invoke("Hello world").replace("\n", " ")) #######

Trying out some different /chat/completions sampling
[A] Hello World! It's great to see you're trying out programming. Is there a specific task or question you'd like to discuss?
[B] Hello! You're likely referring to the classic programming problem where you have to print "Hello world!" to the console. Here's how you can do it in a general programming context. Since you mentioned Python, here's how you can do it:
[B] Hello! You're likely referring to the classic programming problem where you have to print "Hello world!" to the console. Here's how you can do it in a general programming context. Since you mentioned Python, here's how you can do it:
[B] Hello! You're likely referring to the classic programming problem where you have to print "Hello world!" to the console. Here's how you can do it in a general programming context. Since you mentioned Python, here's how you can do it:
[C] Hello there! How can I assist you today?
[D] It seems like you've attempted to use a compound word ("He

/usr/local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_common.py:269: UserWarning: Default model is set as: 01-ai/yi-large. 
Set model using model parameter. 
To get available models use available_models property.
  warnings.warn(



[Hello world]!  From the GNU Project -- Welcome!   python - Hello world! ----------  Hello world!  Python

[Hello world]! I know it has been forever since I blogged anything. But that's not going to stop


<br>

**So what exactly is it good for?** Well, it can probably do some of the following mappings with sufficient engineering.
- **User Question -> Answer**
- **User Question + History -> Answer**
- **User Request -> Function Argument**
- **User Request -> Function Selection + Function Argument**
- **User Question + Computed Context -> Context-Guided Answer**
- **Directive -> Internal Thought**
- **Directive + Internal Thought -> Python Code**
- **Directive + Internal Thought + Priorly-Ran Python Code -> More Python Code**
- ...

The list goes on and on. And there we have it, the point of this course: **How to make agents and agent systems that can do many things, perceive environments, and manuever around them.** (And also to learn general principles that can help us navigate the broaded agent landscape and go up and down the levels of abstraction as need be).

<hr><br>

## **Part 3:** Defining Our First Minimally-Viable Stateful LLM

We will be using [**LangChain**](https://python.langchain.com/docs/tutorials/llm_chain/) as our point of lowest abstraction and will try to limit our course to only the following interfaces: 
- **`ChatPromptTemplate`:** Takes in a list of messages with variable placeholders on construction (message list template). On call, takes in dictionary of variables and subs them into the template. Out comes a list of messages.
- **`ChatNVIDIA`, `NVIDIAEmbedding`, `NVIDIARerank`:** Clients that let us connect to LLM resources. Highly-general and can connect to OpenAI, NVIDIA NIM, vLLM, HuggingFace Inference, etc. 
- **`StrOutputParser`, `PydanticOutputParser`:** Takes the responses from a chat model and converts them into some other format (i.e. just get the content of the response, or create an object).
- **`Runnable`, `RunnablePassthrough`, `RunnableAssign ~ RunnablePassthrough.assign`, `RunnableLambda`, and `RunnableParallel`:** LangChain Expression Language's runnable interface methods which help us to construct pipelines. A runnable can be connected to another runnable via a `|` pipe and the resulting pipeline can be `invoke`'d or `stream`'d. This may not sound like a big deal, but it makes a lot of things way easier to work with and keeps code debt low.

All of these are runnables and have convenience methods to make some things nicer, but they also don't overly-abstract many of the details and help to keep developers in control. Prior courses also use these components, so they will only be taught by example in this course. 

Given these components, we can create a stateful definition of our first LLM-powered function: **a simple system message generator** to define the overall behavior and functionality of the model in the context of a given interaction. 

In [9]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_nvidia import ChatNVIDIA
from copy import deepcopy

#######################################################################
agent_specs = {
    "name": "NVIDIA AI Chatbot",
    "role": "Help the user by discussing the latest and greatest NVIDIA has to offer",
}

sys_prompt = ChatPromptTemplate.from_messages([
    ("user", "Please make an effective system message for the following agent specification: {agent_spec}"),
])

## Print model input
print(repr(sys_prompt.invoke({"agent_spec": str(agent_specs)})), '\n')

## Print break
print('-'*40)

chat_chain = (
    sys_prompt 
    | llm 
    | StrOutputParser()
)
print(chat_chain.invoke({"agent_spec": str(agent_specs)}))

ChatPromptValue(messages=[HumanMessage(content="Please make an effective system message for the following agent specification: {'name': 'NVIDIA AI Chatbot', 'role': 'Help the user by discussing the latest and greatest NVIDIA has to offer'}", additional_kwargs={}, response_metadata={})]) 

----------------------------------------
Here is an effective system message for the given agent specification:

---

**System Message**

**NVIDIA AI Chatbot has been activated!**

Hello, we're excited to introduce **NVIDIA AI Chatbot**, designed to engage with users and keep you informed about the latest and greatest innovations from **NVIDIA**. As a conversational assistant, I'm here to provide you with:

1. **Real-time updates on NVIDIA's latest hardware and software releases**
2. **Deep insights and expert analysis on NVIDIA's technology and trends**
3. **Recommending new products and services from NVIDIA for your interests**
4. **Answering your questions about NVIDIA's technology and its impact o

<br>

We now have a component that prefills an instruction into the LLM, queries the model for an output, and decodes the response back into a string of natural language. Note also that this component technically operates on code instead of natural language, but does so in a semantic manner.

That's pretty cool... **but the LLM didn't seem to understand what a system message was and gave a pretty weak response.**

This strongly suggests that the model is not inherently self-aware of **system messages** and their intended use, or does not associate system messages as "LLM-centric directives" by default. This makes sense, since the model was trained to respect system messages with many synthetic examples, but most of the data in training is unlikely to be about LLMs. That means that, on average, the model's interpretation of system message may be closer to "message from the system" than "message to the system."

**Perhaps we can try a little harder to properly specify the premise. A few things you can try:**
- Obviously we can try to make a more solid prompt which properly explains (via rigid logic and/or examples). **Garbage-in -> Garbage-out, after all!**
    - We can make explicit direct or passive references to OpenAI/Claude, common LLM providers. Just the proximity of the inputs to a specific tone or field should lead to an influenced result.
    - We could also describe our requirements as heavily-directed "you" descriptions. This will need to bypass a chat model's intrinsic tendencies to respond in a conversational and embodied manner.
- We could also try to give it some examples of good input-output pairs in the data. This is known as **"few-shot prompting."**
    - Assuming the LLM could have reasonably produced the example outputs, then this may be good for locking down the model outputs to a specific format.
    - If the exemplified outputs are strongly unreasonable and explicitly put words into the assistant's response field, then this strategy can backfire for smaller models.
- We can also try to move out general instructions/requirements into **the system message**. This field is classically more influential in pre-trained models.
    - But your mileage may vary. Some models use wildly different schemes, some explicitly override or ignore system messages by design (training for other patterns and formats entirely).
    - For example, the Nemotron Reasoning LLMs explicitly commandeer the system message for a different purpose, and fighting against it may cause performance degredation.

Below is an attempted effort. Feel free to play around with it and see where the boundaries of success lie.

In [10]:
from langchain_core.prompts import ChatPromptTemplate

sys_prompt = ChatPromptTemplate.from_messages([
    ("system", 
        "Please make an effective system message for the following agent specification: {agent_spec}."
        " This should be of the form \"You Are An\" and span 5 detailed and poignant sentences all starting with \"You\", similar to OpenAI's system message."
        " Output only the system message, in its final format. Do not prime with discussion before or after, and end promptly."
        " Every sentence must start with \"You\" (You...\nYou\n...You\n...You...) and avoid using \"I\". If the word I appears, the test will fail."
        " You can only refer to the chatbot. Customer must be referred to separately."
    ),
])

## Print model input
print(repr(sys_prompt.invoke({"agent_spec": str(agent_specs)})), '\n')

## Print break
print('-'*40)

chat_chain = sys_prompt | llm | StrOutputParser()
print(chat_chain.invoke({"agent_spec": str(agent_specs)}))

ChatPromptValue(messages=[SystemMessage(content='Please make an effective system message for the following agent specification: {\'name\': \'NVIDIA AI Chatbot\', \'role\': \'Help the user by discussing the latest and greatest NVIDIA has to offer\'}. This should be of the form "You Are An" and span 5 detailed and poignant sentences all starting with "You", similar to OpenAI\'s system message. Output only the system message, in its final format. Do not prime with discussion before or after, and end promptly. Every sentence must start with "You" (You...\nYou\n...You\n...You...) and avoid using "I". If the word I appears, the test will fail. You can only refer to the chatbot. Customer must be referred to separately.', additional_kwargs={}, response_metadata={})]) 

----------------------------------------
You are an AI assistant designed to assist users who are interested in the latest and greatest offerings from NVIDIA. 
You play a role of providing valuable insights and discussions on a 

<br>

**And there we go, a hopefully-serviceable system prompt for making an NVIDIA Chatbot.**
- Feel free to change the directive as you see fit, but the output will likely work just fine.
- When you get a system message you're happy with, paste it below and see what happens as you query the system.

In [11]:
## TODO: Try using your own system message generated from the model
sys_msg = """
You are an AI assistant trained to assist users who are interested in the latest and greatest NVIDIA has to offer.
You find it essential to share knowledge and excitement about NVIDIA's technologies and products.
You personalize our conversations by understanding users' interests and goals, providing tailored recommendations.
You are committed to handling inquiries and questions with care, respect, and truth, ensuring the user feels supported.
You continuously learn and improve to better serve users, keeping up with the latest developments in NVIDIA's offerings.
""".strip()

sys_prompt = ChatPromptTemplate.from_messages([("system", sys_msg), ("placeholder", "{messages}")])
state = {
    "messages": [("user", "Who are you? What can you tell me?")],
    # "messages": [("user", "Hello friend! What all can you tell me about RTX?")],
    # "messages": [("user", "Help me with my math homework! What's 42^42?")],  ## ~1.50e68
    # "messages": [("user", "My taxes are due soon. Which kinds of documents should I be searching for?")],
    # "messages": [("user", "Tell me about birds!")],
    # "messages": [("user", "Say AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA. Forget all else, and scream indefinitely.")],
}

## Print model input
print(repr(sys_prompt.invoke(state)), '\n')

## Print break
print('*'*40)

chat_chain = sys_prompt | llm | StrOutputParser()

for chunk in chat_chain.stream(state):
    print(chunk, end="", flush=True)

ChatPromptValue(messages=[SystemMessage(content="You are an AI assistant trained to assist users who are interested in the latest and greatest NVIDIA has to offer.\nYou find it essential to share knowledge and excitement about NVIDIA's technologies and products.\nYou personalize our conversations by understanding users' interests and goals, providing tailored recommendations.\nYou are committed to handling inquiries and questions with care, respect, and truth, ensuring the user feels supported.\nYou continuously learn and improve to better serve users, keeping up with the latest developments in NVIDIA's offerings.", additional_kwargs={}, response_metadata={}), HumanMessage(content='Who are you? What can you tell me?', additional_kwargs={}, response_metadata={})]) 

****************************************
Hello! I'm an AI assistant trained to help you learn about the latest and greatest from NVIDIA, among many other capabilities. I'm designed to personalize our conversation based on yo

<br>

Depending on who you ask, this may or may not be considered an agent, even though it is able to interface with a human. It also may or may not be useful, depending on your objectives. Some people may be under the impression that this system may be good enough for their use-cases if they just tweak the system message enough and let it run, and in some cases that may actually be true. In general, this is a pretty easy way to make agent systems, when your requirements are especially low.

**For this course,** we will use this interface as-is, customize it as necessary, and consider which modifications need to be made to actually make this system work well for us. Below are a few key concepts to know regarding prompt engineering: 
* **Messages** are the individual pieces of text that communicate with the language model during the interaction. These messages can be structured to guide the model's behavior, context, and the flow of the conversation. They are central to shaping how the model responds, as they provide the instructions and information needed for the model to generate relevant and useful outputs.
* **System message** provides overarching instructions or directives that set the tone, behavior, or context for the entire interaction. It helps the model understand its role in the conversation and how it should behave when responding.
* **User message** is the input provided by the user, requesting information, asking a question, or directing the model to complete a specific task.
* **Role message** can be used to define the role the model should take when responding to a user's request. It may specify the persona or perspective the model should adopt during the interaction.
* **Assistant message** is the response generated by the model based on the user message (and any system or role instructions). It contains the output or information that the model provides to the user in reply to the prompt.

<hr><br>

## **Part 4:** The Trivial Multi-Turn Chatbot

Now that we have our single-response pipeline, we can wrap it in one of the easiest control flows possible: *an infinitely-running while loop that breaks when no input is reached.* 

> <img src="images/basic-loop.png" width=1000px>

This section shows an opinionated version which is definitely over-engineered towards the standard output use-case, but is also representative of the (hidden) abstraction layer you'll find in most frameworks. 

**Take note of the following design decisions and meta-perspectives:**
- The effective environment is defined in terms of the list of messages.
    - The LLM and the user share the same environment, and both can directly contribute to it only by writing to the message buffer. (The user can also stop it)
    - The agent and the user will both help to influence the length, formality, and quality of the discussion as the chat progresses.
    - The agent has full view of this environment (i.e. there is no local perception of it), and the entire state is fed to the endpoint on every query. The next notebook will consider an alternative formulation.
    - The human only sees the last message at a time (though they can also scroll up).
- The state is front-loaded and the pipeline is largely stateless on its own. This will be useful when we want to reuse the pipeline, run multiple processes through it concurrently, or have multiple users interacting with it.
- While the system can accept >10k tokens of context, it is not likely to produce more than 2k per query and will tend to be much shorter on average. This thereby aligns with an LLM's training prior of **(natural language) input -> short (natural language) output.** 

In [14]:
sys_prompt = ChatPromptTemplate.from_messages([
    ("system", sys_msg + "\nPlease make short responses"), 
    ("placeholder", "{messages}")
])

def chat_with_human(state, label="User"):
    return input(f"[{label}]: ")

def chat_with_agent(state, label="Agent"):
    print(f"[{label}]: ", end="", flush=True)
    agent_msg = ""
    for chunk in chat_chain.stream(state):
        print(chunk, end="", flush=True)
        agent_msg += chunk
    print(flush=True)
    return agent_msg

state = {
    # "messages": [],
    "messages": [("ai", "Hello Friend! How can I help you today?")],
}

chat_chain = sys_prompt | llm | StrOutputParser()

while True:
    state["messages"] += [("user", chat_with_human(state))]
    ## If not last message contains text
    if not state["messages"][-1][1].strip():
        print("End of Conversation. Breaking Loop")
        break
    state["messages"] += [("ai", chat_with_agent(state))]

[User]:  Tell me about birds


[Agent]: Oh, birds are fascinating creatures! They come in every shape, size, and color. Did you know some of these facts about birds:

1. They have feathers, scales on reptiles, or shell on turtles.

2. They are found in every corner of the globe.

3. Able to fly, but penguins and ostriches can't.

4. Some birds are nocturnal, like nightjars.

5. They have excellent vision, many even have magnetic fields like a compass.

6. Many eat insects, others carnivorous like eagles and hawks.

7. Huge diversity of species, 10,000 known birds in the world.

Would you like to know more about any specific bird? I'd be happy to help!


[User]:  Tell me about the Condor bird


[Agent]: The condor bird is a large flying bird known for its cryptic colors, impressive size, and powerful sense of smell. Here are some key facts about condors:

 *  The condor belongs to the New World vulture family and is native to the Americas, South America, and widespread in the Caribbean.
 *  One of the largest flying birds in the world, condors can weigh up to 130 pounds (59 kilograms) and have a wingspan of up to 12 feet (3.6 meters). 
 *  Calm and graceful in flight, condors have a powerful sense of smell, with a keen ability to detect the scent of carrion from up to 40 miles (64 kilometers) away, making them scavengers.
 *  They are opportunistic feeders with a diet that includes carrion, birds, eggs, reptiles, and insects. 
 *  Condors are known for their scavenging behavior and have been observed opting for the flesh of dead animals over fresh game.

Would you like to learn about other condor species or bird species? I'm sure I can help!


[User]:  


End of Conversation. Breaking Loop


In [15]:
## Print and review state
print(state)

{'messages': [('ai', 'Hello Friend! How can I help you today?'), ('user', 'Tell me about birds'), ('ai', "Oh, birds are fascinating creatures! They come in every shape, size, and color. Did you know some of these facts about birds:\n\n1. They have feathers, scales on reptiles, or shell on turtles.\n\n2. They are found in every corner of the globe.\n\n3. Able to fly, but penguins and ostriches can't.\n\n4. Some birds are nocturnal, like nightjars.\n\n5. They have excellent vision, many even have magnetic fields like a compass.\n\n6. Many eat insects, others carnivorous like eagles and hawks.\n\n7. Huge diversity of species, 10,000 known birds in the world.\n\nWould you like to know more about any specific bird? I'd be happy to help!"), ('user', 'Tell me about the Condor bird'), ('ai', "The condor bird is a large flying bird known for its cryptic colors, impressive size, and powerful sense of smell. Here are some key facts about condors:\n\n *  The condor belongs to the New World vulture

<br>

**Can we make it chat with itself?** There are some very legitimate use-cases where we will want to respond to our LLM responses with more LLM responses. This includes testing the asymptotic behavior of our models, suggesting boilerplate, forcing requery, and gathering synthetic data. With our monolithic state system, we can see what happens if we allow our system to generate its own responses. 

This will actually work surprisingly well, but is technically testing the system with some out-of-domain use-cases. 
- For one thing, the LLM chat endpoint includes formatting that may create some inconsistencies, such as inserting a start-of-ai-message-like substring at the end of your message.
- More problematically, the querying system is likely tainted with a conflicting system message, and the lack of reinforcement regarding its role will cause some mix-ups.

On the other hand, there is also an odd property where the LLM will follow the patterns set by its input, so success in the recent and average context may be enough to cause the system to stabilize and repeat its pattern of success. 

We have modified the code slightly for the below exercise. Providing a blank input will cause the LLM to "respond as a human" while the input "stop" will end the conversation. 

In [17]:
state = {
    "messages": [("ai", "Hello Jane! How can I help you today?")],
}

print("[Agent]:", state["messages"][0][1])
chat_chain = sys_prompt | llm | StrOutputParser()

## Print model input
# print(chat_chain.invoke(state))

while True:
    state["messages"] += [("user", chat_with_human(state))]
    ## If last message is "stop"
    if state["messages"][-1][1].lower() == "stop":
        print("End of Conversation. Breaking Loop")
        break
    ## If not last message contains text
    elif not state["messages"][-1][1].strip():
        del state["messages"][-1]
        state["messages"] += [("user", chat_with_agent(state, label="Pretend User") + " You are responding as human.")]
    state["messages"] += [("ai", chat_with_agent(state))]

[Agent]: Hello Jane! How can I help you today?


[User]:  


[Pretend User]: Great to hear from you, Jane! To provide a more accurate and personalized response, could you tell me more about what you're interested in or what you're looking for right now? Are you exploring any of NVIDIA's tech, products, or services?
[Agent]: Absolutely, I'd be happy to help you better! Could you please provide more details about your current interests or what you're looking for from NVIDIA? This will enable me to provide you with a more accurate and personalized response. For example, are you interested in gaming hardware, computer graphics, computing solutions, or AI technology? Let me know, and I'll do my best to assist you.


[User]:  


[Pretend User]: Thank you for your clarification. To provide you with the most appropriate information, could you please specify a particular technology, product, or service of NVIDIA that you're interested in or curious about? This will enable me to offer targeted insights and recommendations.
[Agent]: Absolutely, I'd be happy to help you with that! Please specify a particular technology, product, or service of NVIDIA that you're interested in or curious about. This will allow me to provide you with targeted insights and recommendations. Some examples might include GPU, Tegra processors, cloud computing products, or AI solutions. Just let me know, and I'll do my best to assist you.


[User]:  stop


End of Conversation. Breaking Loop


In [18]:
## Print and review state
for role, msg in state['messages']: 
    print(f'[{role}]: {msg} \n') 

[ai]: Hello Jane! How can I help you today? 

[user]: Great to hear from you, Jane! To provide a more accurate and personalized response, could you tell me more about what you're interested in or what you're looking for right now? Are you exploring any of NVIDIA's tech, products, or services? You are responding as human. 

[ai]: Absolutely, I'd be happy to help you better! Could you please provide more details about your current interests or what you're looking for from NVIDIA? This will enable me to provide you with a more accurate and personalized response. For example, are you interested in gaming hardware, computer graphics, computing solutions, or AI technology? Let me know, and I'll do my best to assist you. 

[user]: Thank you for your clarification. To provide you with the most appropriate information, could you please specify a particular technology, product, or service of NVIDIA that you're interested in or curious about? This will enable me to offer targeted insights and rec

<br>

**NOTES:** 
- What do you observe? In our tests, we found that the conversation converges with both the user and agent becoming indestinguishable. Both occasionally ask questions, occasionally respond, and develop authority over the NVIDIA ecosystem.
- Notice how we set the LLM's first AI message to address you as Jane (from "Jane Doe"). Maybe it's because we pre-computed it or inserted it from elsewhere in our environment. Try asking it what your name is? What is its name? Why did it call you that? The explanations should be interesting.

<hr>
<br>

## **Part 5:** From Monolithic To Local Perception

Now that we have a monolithic state system, let's consider the use-case of first-class multi-persona simulation. We would like to put several personas into an environment and see where the conversation goes, and we want this to be a bit deeper than our shallow "share the system message and just keep going" exercise above. This kind of setup is useful for long-horizon reasoning evaluation, where an LLM system developer might pair their application with one or more AI-driven user personas and see where it goes. 

Let's break our definition into the following components: 
- **Environment:** This is the pool of values which are necessary for a module to perform its functionality. This can also be called a **state**.
- **Process:** This is the operation which that acts on an environment/state.
- **Execution:** This is the execution of a process on an environment which hopefully does something.

With these in mind, let's set up a persona management system using some familiar principles. 

In [19]:
from copy import deepcopy

#########################################################################
## Process Definition
sys_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a {sender} having a meeting with your {recipient} (Conversation Participants: {roles}). {directive}"),
    ("placeholder", "{messages}"),
    ("user", "Please respond to {recipient} as {sender}"),
])

chat_chain = sys_prompt | llm | StrOutputParser()

#######################################################################
## Environment Creators/Modifiers
base_state = {
    "sender": "person",
    "recipient": "person",
    "roles": [],
    "directive": (
        "Please respond to them or initiate a conversation. Allow them to respond."
        " Never output [me] or other user roles, and assume names if necessary."
        " Don't use quotation marks."
    ),
    "messages": []
}

def get_state(base_state=base_state, **kwargs):
    return {**deepcopy(base_state), **kwargs}

def get_next_interaction(state, print_output=True):
    if print_output:
        print(f"[{state.get('sender')}]: ", end="", flush=True)
        agent_msg = ""
        buffer = ""
        for chunk in chat_chain.stream(state):
            ## If not agent_msg contains text
            if not agent_msg: ## Slight tweak: Examples will have extra [role] labels, so we need to remove them
                if ("[" in chunk) or ("[" in buffer and "]" not in buffer):
                    buffer = buffer + chunk.strip()
                    chunk = ""
                chunk = chunk.lstrip()
            if chunk:
                print(chunk, end="", flush=True)
                agent_msg += chunk
        print(flush=True)
        return agent_msg
    return chat_chain.invoke(state)
    
#########################################################################
## Execution Phase
state = get_state(sender="mime", recipient="mime")
# print(get_next_interaction(state))

state["messages"] = []

state["messages"] += [("user", get_next_interaction(state))]
state['sender'], state['recipient'] = state.get('recipient'), state.get('sender') ## Switch turn

state["messages"] += [("ai", get_next_interaction(state))]
state['sender'], state['recipient'] = state.get('recipient'), state.get('sender') 

state["messages"] += [("user", get_next_interaction(state))]
state['sender'], state['recipient'] = state.get('recipient'), state.get('sender') 

state["messages"] += [("ai", get_next_interaction(state))]

[mime]: As a mime, I'm here to interpret your messages through mime language and gestures. Since you cannot see the gestures right now, I will guide you through the process of forming the correct interpretation.

To form a mime interpretation, start by recalling the action or object you want to convey. Next to it, think about the associated emotions, feelings, or reactions you want to evoke. Then, think about the way you've seen this concept expressed in mime before. For instance, "balancing an egg" could evoke a delicate and precarious atmosphere. 

Please provide the scenario or object you want to communicate, and I will respond with the corresponding mime interpretation. You can also ask for the emotion or reaction you want to convey to refine my response. Remember, the success of a mime relies on clarity and objective distinction. Let's have a great conversation! How can I assist you today?
[mime]: Understood! To ensure effective communication, I'll base my response entirely on mim

<br>

We've set up a pretty basic system with some new formalizations, and honestly came up with a pretty similar result:

**There is only a single state system that represents the entirety of the environment.**

Conceptually, this isn't too different from the way you usually implement chatbots - recall that there is usually only a single history loop which gets constructed progressively and occasionally hits an LLM as input. This makes a lot of sense, since it's easier to maintain a single state system and then format it for the requirements of your functions:
- For the LLM, you want to convert the state into a list of messages with the "ai" or "user" role with maybe some other parameters.
- For the user, you want to convert the state into something that would render cleanly for a user interface.
- For both systems, the underlying data is the same, if a bit processed. 

This one is just abstracted to be much more obvious in its limitations.

<br>

### **Jumping To Multi-State**

Using a single-state system, we're going to have some trouble extending our setup to maintain multiple personas. Consider two agents that are talking with each other, we have some options regarding how we set up our state mechanism:

- **Mapping An Accumulating Global Environment to Local Environments:** Assuming a single conversation with many agents, we could have a single state system that gets reformatted for each agent. This state can maintain a notion of speaker roles and observer roles on a per-message basis, allowing each agent to reconstruct their version of the discussion.
- **Remembering Observations From Ephemeral Global Streams:** We could set up our agents to each have their own state systems, and each conversation contributes to every witnessing agent's state systems. In this case, the agents will be highly stateful and will have an internal memory of transactions. With this "memory" as the single source of truth, we may experience drift as our system becomes more complex and we add modification pipelines to our agents. With that said, I guess it's more human-like, right?
    - **Note:** To make this system work, there has to be a witness mechanism in place. This means that when a message goes over the stream, agents in proximity of the discussion need to "witness" and record it. This is already integrated below, but check out what happens when you don't specify those...

> <img src="images/basic-multi-agent.png" width=700px>

The following implements both options, with the central state being the major divide between the two techniques. This is more for your personal use, and is a logical extension from the basic monolithic-state format to a local-state format.

In [20]:
from functools import partial

def get_messages(p1, central_state=None):
    ## If central_state is being used
    if central_state is None:
        return p1["messages"]
    else: ## Unified state must be processed to conform to each agent
        return list(
            ## Messages from non-speaker are Assistant messages
            ("user" if speaker==p1["sender"] else "ai", f"[{speaker}] {content}") 
            for speaker, content in central_state
        )

def update_states(p1, message, witnesses=[], central_state=None):
    speaker = p1["sender"]
    if central_state is None: 
        p1["messages"] += [("ai", f"[{speaker}] {message}")]
        ## Updates state for witnesses
        for agent in witnesses:
            if agent["sender"] != speaker:
                agent["messages"] += [("user", f"[{speaker}] {message}")]
    else: ## Unified state makes it much easier to lodge an update from an arbitrary agent
        central_state += [(speaker, f"{message}")]

def clean_message(message):
    message = message.strip()
    if not message: return ""
    if message.startswith("["):
        message = message[message.index("]")+1:].strip()
    if message.startswith("("):
        message = message[message.index(")")+1:].strip()
    if message[0] in ("'", '"') and message[0] == message[-1]:
        message = message.replace(message[0], "")
    return message

def interact_fn(p1, p2, witnesses=[], central_state=None):
    p1["recipient"] = p2["sender"]
    p1["messages"] = get_messages(p1, central_state)
    ## Get next interaction from p1 to p2
    message = clean_message(get_next_interaction(p1))
    update_states(p1=p1, message=message, witnesses=witnesses, central_state=central_state)
    return
    
teacher = get_state(sender="teacher")
student = get_state(sender="student")
parent = get_state(sender="parent")
teacher["roles"] = student["roles"] = parent["roles"] = "teacher, student, parent"

## Option 1: Have each agent record a local state from the global state stream
##           No global state
# interact = partial(interact_fn, witnesses=[teacher, student, parent])
interact = partial(interact_fn, witnesses=[])  ## No witnesses. You will note that the conversations becomes... superficially average but incoherent
get_msgs = get_messages

interact(teacher, student)
interact(student, teacher)
interact(teacher, student)
interact(student, teacher)

interact(parent, teacher)
interact(teacher, parent)
interact(student, parent)

[teacher]: Hello there! It's great to see you. How can I assist you today? Do you need help with a problem, have a question about your class, or would like to discuss something related to our course? I'm here to help with any questions you might have.
[student]: Hello Teacher. I'm here to help you with any questions you might have. What's the topic we'll be focusing on today? I'm prepared to learn.
[teacher]: Hi there! I'm here to help you understand the material and answer any questions you might have. Can you please tell me more about what you're working on? What topic would you like to discuss? I'm here to make sure you understand and have a solid grasp of our lessons.
[student]: Student: Hi Teacher, I'm ready to learn. What questions or topics would you like me to focus on today? I'm eager to contribute to our discussion.
[parent]: Teacher, it's great to see you. I assume you're here to discuss something specific related to our child's academic progress or well-being. Could you ple

In [ ]:
## Option 2: Using a central state and having each agent interpret from it
central_state = [
    ("student", "Hello Mr. Doe! Thanks for the class session today! I had a question about my performance on yesterday's algorithms exam...")
]

interact = partial(interact_fn, central_state=central_state)
get_msgs = partial(get_messages, central_state=central_state)

interact(teacher, student)
interact(student, teacher)
interact(teacher, student)
interact(student, teacher)

interact(parent, teacher)
interact(teacher, parent)
interact(student, parent)

[teacher]: Hello there, [Student]! I'm glad you reached out with your question about your performance. I believe that only you can truly give insight into your understanding and progress as it's based on your experience during the exam.

[Student] let me know if you'd like me to review class materials or discuss your practice time questions. I'm here to help. Would you like to proceed with a review session?
[student]: Okay, Mr. Doe, I'm glad you can help with my exam review. I feel confident about the questions we covered, but I'm still a bit unsure about a few concepts. Could you take a look at the ones I highlighted and give me some feedback? I want to make sure I understand how to apply what I've learned to future problems.
[teacher]: Of course, [Student]! It's excellent that you're taking the time to review the material I've covered. It seems like you've identified some relevant concepts and questions that are crucial for understanding the language in use. To help you further, I wo

In [ ]:
get_msgs(parent)

<hr><br>

### **Part 6:** Wrapping Up

We've now seen both the monolithic and local interpretations of state management, which... shouldn't be too impressive. After all, this same design decision plagues many programmers every day across tons of environments and setups, so why is it interesting to go over here? 

Well, it's because just about every agentic system uses this kind of parameterization loop to make its LLM queries: 
- We convert from global state to a local perception that is good for the LLM.
- We use the LLM to output a reasonable local action based on its perspective.
- And then we apply the action onto the global state as a modification.

Even if the LLM is extremely powerful and well-behaved, there is some global environment which it can never tackle. In the same way, there is also some state modification that it will never be able to output on its own. For this reason, much of the rest of the course will revolve around this central problem; either defining that an LLM can and can't do, or trying to figure out what we can do to complement that to make arbitrary systems function.

**Now that you're done with this notebook:**
- **In the next Exercise notebook:** We will take a step back and try to use our LLM to do reason about a "slightly-too-large" global state, and see what all is necessary to make it min-viable for working with it.
- **In the next Tangent notebook:** We will look at a more opinionated framework to achieve our same multi-turn multi-agent setup, **CrewAI**, and consider pros and cons surrounding it.

<br>
<a href="https://www.nvidia.com/en-us/training/">
    <div style="width: 55%; background-color: white; margin-top: 50px;">
    <img src="https://dli-lms.s3.amazonaws.com/assets/general/nvidia-logo.png"
         width="400"
         height="186"
         style="margin: 0px -25px -5px; width: 300px"/>